## Trying small requests

Once I have all of the data I need I can join all data frames and calculate fields needed, then I can upscale to deal with all api requests (will take a while)

In [1]:
import requests
import pandas as pd
from etl.extract.extract import extract_tickers
import numpy as np

In [2]:
test_ticker_list = ['AAPL', 'AACB', 'AAPD', 'ABLLL', 'HPKEW', 'HPK', 'NMRA', 'PLSE', 'PNTG', 'SEEM']
API_KEY = 'b95993f82e2d0048935cfd947df28088'


In [3]:
# get data from financial score endpoint
score_results = []
for tick in test_ticker_list:
    url = f'https://financialmodelingprep.com/api/v4/score?symbol={tick}&apikey={API_KEY}'
    try:
        res = requests.get(url)
        data = res.json()
        if data:
            score_results.append(data[0])
    except Exception as e:
        print(f'Error fetching {tick}: {e}')
    
print(score_results)

[{'symbol': 'AAPL', 'altmanZScore': 9.097169228144661, 'piotroskiScore': 8, 'workingCapital': -25897000000, 'totalAssets': 331233000000, 'retainedEarnings': -15552000000, 'ebit': 127364000000, 'marketCap': 2987745469000, 'totalLiabilities': 264437000000, 'revenue': 400366000000}, {'symbol': 'ABLLL', 'altmanZScore': 1.2203494803590922, 'piotroskiScore': 2, 'workingCapital': 97098567, 'totalAssets': 874164752, 'retainedEarnings': -57896606, 'ebit': -2357387, 'marketCap': 796285856, 'totalLiabilities': 450870080, 'revenue': 112780530}, {'symbol': 'HPKEW', 'altmanZScore': 0.8090221975706576, 'piotroskiScore': 7, 'workingCapital': -146265000, 'totalAssets': 3093081000, 'retainedEarnings': 466595000, 'ebit': 257515000, 'marketCap': 106778749, 'totalLiabilities': 1459687000, 'revenue': 1039098000}, {'symbol': 'HPK', 'altmanZScore': 1.2988716040530062, 'piotroskiScore': 7, 'workingCapital': -146265000, 'totalAssets': 3093081000, 'retainedEarnings': 466595000, 'ebit': 257515000, 'marketCap': 12

In [4]:

score_df = pd.DataFrame(score_results)
score_df = score_df[['symbol', 'ebit', 'workingCapital']]

In [5]:
#Get balance sheet info
balance_sheet_res = []
for tick in test_ticker_list:
    url = f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{tick}?period=annual&apikey={API_KEY}'
    try:
        res = requests.get(url)
        data = res.json()
        if data:
            balance_sheet_res.append(data[0])
    except Exception as e:
        print(f'Error fetching {tick}: {e}')
        
print(balance_sheet_res)


[{'date': '2024-09-28', 'symbol': 'AAPL', 'reportedCurrency': 'USD', 'cik': '0000320193', 'fillingDate': '2024-11-01', 'acceptedDate': '2024-11-01 06:01:36', 'calendarYear': '2024', 'period': 'FY', 'cashAndCashEquivalents': 29943000000, 'shortTermInvestments': 35228000000, 'cashAndShortTermInvestments': 65171000000, 'netReceivables': 66243000000, 'inventory': 7286000000, 'otherCurrentAssets': 14287000000, 'totalCurrentAssets': 152987000000, 'propertyPlantEquipmentNet': 45680000000, 'goodwill': 0, 'intangibleAssets': 0, 'goodwillAndIntangibleAssets': 0, 'longTermInvestments': 91479000000, 'taxAssets': 19499000000, 'otherNonCurrentAssets': 55335000000, 'totalNonCurrentAssets': 211993000000, 'otherAssets': 0, 'totalAssets': 364980000000, 'accountPayables': 68960000000, 'shortTermDebt': 22511000000, 'taxPayables': 26601000000, 'deferredRevenue': 8249000000, 'otherCurrentLiabilities': 50071000000, 'totalCurrentLiabilities': 176392000000, 'longTermDebt': 96548000000, 'deferredRevenueNonCurre

In [6]:
balance_sheet_df = pd.DataFrame(balance_sheet_res)
balance_sheet_df = balance_sheet_df[['date', 'symbol', 'cashAndCashEquivalents',
                  'propertyPlantEquipmentNet', 'totalDebt']]

In [7]:
#Get company info
company_info_res = []
for tick in test_ticker_list:
    url = f'https://financialmodelingprep.com/api/v3/profile/{tick}?apikey={API_KEY}'
    try:
        res = requests.get(url)
        data = res.json()
        if data:
            company_info_res.append(data[0])
    except Exception as e:
        print(f'Error fetching {tick}: {e}')
        
print(company_info_res)

[{'symbol': 'AAPL', 'price': 212.26, 'beta': 1.275, 'volAvg': 60718767, 'mktCap': 3170272908000, 'lastDiv': 1.01, 'range': '169.21-260.1', 'changes': 1.47, 'companyName': 'Apple Inc.', 'currency': 'USD', 'cik': '0000320193', 'isin': 'US0378331005', 'cusip': '037833100', 'exchange': 'NASDAQ Global Select', 'exchangeShortName': 'NASDAQ', 'industry': 'Consumer Electronics', 'website': 'https://www.apple.com', 'description': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, vi

In [8]:
company_info_df = pd.DataFrame(company_info_res)
company_info_df = company_info_df[['symbol', 'mktCap', 'industry', 'sector',
                 'image', 'ipoDate',
                 'isActivelyTrading']]

In [9]:
test_merge = score_df.merge(balance_sheet_df, on='symbol', how='outer')
test_merge = test_merge.merge(company_info_df, on='symbol', how='outer')
test_merge

,symbol,ebit,workingCapital,date,cashAndCashEquivalents,propertyPlantEquipmentNet,totalDebt,mktCap,industry,sector,image,ipoDate,isActivelyTrading
0,AACB,NaN,NaN,NaN,NaN,NaN,NaN,275919740,Shell Companies,Financial Services,https://images.financialmodelingprep.com/symbo...,2025-02-13,True
1,AAPD,NaN,NaN,NaN,NaN,NaN,NaN,32942306,Asset Management - Leveraged,Financial Services,https://images.financialmodelingprep.com/symbo...,2022-08-10,True
2,AAPL,1.273640e+11,-2.589700e+10,2024-09-28,2.994300e+10,4.568000e+10,1.066290e+11,3170272908000,Consumer Electronics,Technology,https://images.financialmodelingprep.com/symbo...,1980-12-12,True
3,ABLLL,-2.357387e+06,9.709857e+07,2024-12-31,1.319443e+08,5.747639e+06,3.859139e+08,842259617,Insurance - Life,Financial Services,https://images.financialmodelingprep.com/symbo...,2023-11-24,True
4,HPK,2.575150e+08,-1.462650e+08,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,1306684455,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-29,True
5,HPKEW,2.575150e+08,-1.462650e+08,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,106778749,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-30,True
6,NMRA,-2.743260e+08,2.264820e+08,2024-12-31,1.421480e+08,2.917000e+06,1.853000e+06,108403456,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2023-09-15,True
7,PLSE,-5.622500e+07,1.093940e+08,2024-12-31,1.180380e+08,8.323000e+06,8.898000e+06,1135919802,Medical - Instruments & Supplies,Healthcare,https://images.financialmodelingprep.com/symbo...,2016-05-18,True
8,PNTG,3.748500e+07,2.115600e+07,2024-12-31,2.424600e+07,3.138820e+08,2.730910e+08,1021661160,Medical - Care Facilities,Healthcare,https://images.financialmodelingprep.com/symbo...,2019-10-01,True


In [ ]:
# calculate fields for ranking
# Enterprise Value = market cp + total debt - cash and cash equivalents

test_merge['enterpriseValue'] = test_merge['mktCap'] + test_merge['totalDebt'] - test_merge['cashAndCashEquivalents']
test_merge



,symbol,ebit,workingCapital,date,cashAndCashEquivalents,propertyPlantEquipmentNet,totalDebt,mktCap,industry,sector,image,ipoDate,isActivelyTrading,enterpriseValue
0,AACB,NaN,NaN,NaN,NaN,NaN,NaN,275919740,Shell Companies,Financial Services,https://images.financialmodelingprep.com/symbo...,2025-02-13,True,NaN
1,AAPD,NaN,NaN,NaN,NaN,NaN,NaN,32942306,Asset Management - Leveraged,Financial Services,https://images.financialmodelingprep.com/symbo...,2022-08-10,True,NaN
2,AAPL,1.273640e+11,-2.589700e+10,2024-09-28,2.994300e+10,4.568000e+10,1.066290e+11,3170272908000,Consumer Electronics,Technology,https://images.financialmodelingprep.com/symbo...,1980-12-12,True,3.246959e+12
3,ABLLL,-2.357387e+06,9.709857e+07,2024-12-31,1.319443e+08,5.747639e+06,3.859139e+08,842259617,Insurance - Life,Financial Services,https://images.financialmodelingprep.com/symbo...,2023-11-24,True,1.096229e+09
4,HPK,2.575150e+08,-1.462650e+08,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,1306684455,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-29,True,2.149808e+09
5,HPKEW,2.575150e+08,-1.462650e+08,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,106778749,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-30,True,9.499027e+08
6,NMRA,-2.743260e+08,2.264820e+08,2024-12-31,1.421480e+08,2.917000e+06,1.853000e+06,108403456,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2023-09-15,True,-3.189154e+07
7,PLSE,-5.622500e+07,1.093940e+08,2024-12-31,1.180380e+08,8.323000e+06,8.898000e+06,1135919802,Medical - Instruments & Supplies,Healthcare,https://images.financialmodelingprep.com/symbo...,2016-05-18,True,1.026780e+09
8,PNTG,3.748500e+07,2.115600e+07,2024-12-31,2.424600e+07,3.138820e+08,2.730910e+08,1021661160,Medical - Care Facilities,Healthcare,https://images.financialmodelingprep.com/symbo...,2019-10-01,True,1.270506e+09


In [11]:
#earningsYield
#EBIT/Enterprise value

test_merge['earningsYield'] = (test_merge['ebit'] / test_merge['enterpriseValue']) * 100
test_merge

,symbol,ebit,workingCapital,date,cashAndCashEquivalents,propertyPlantEquipmentNet,totalDebt,mktCap,industry,sector,image,ipoDate,isActivelyTrading,enterpriseValue,earningsYield
0,AACB,NaN,NaN,NaN,NaN,NaN,NaN,275919740,Shell Companies,Financial Services,https://images.financialmodelingprep.com/symbo...,2025-02-13,True,NaN,NaN
1,AAPD,NaN,NaN,NaN,NaN,NaN,NaN,32942306,Asset Management - Leveraged,Financial Services,https://images.financialmodelingprep.com/symbo...,2022-08-10,True,NaN,NaN
2,AAPL,1.273640e+11,-2.589700e+10,2024-09-28,2.994300e+10,4.568000e+10,1.066290e+11,3170272908000,Consumer Electronics,Technology,https://images.financialmodelingprep.com/symbo...,1980-12-12,True,3.246959e+12,3.922563
3,ABLLL,-2.357387e+06,9.709857e+07,2024-12-31,1.319443e+08,5.747639e+06,3.859139e+08,842259617,Insurance - Life,Financial Services,https://images.financialmodelingprep.com/symbo...,2023-11-24,True,1.096229e+09,-0.215045
4,HPK,2.575150e+08,-1.462650e+08,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,1306684455,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-29,True,2.149808e+09,11.978509
5,HPKEW,2.575150e+08,-1.462650e+08,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,106778749,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-30,True,9.499027e+08,27.109617
6,NMRA,-2.743260e+08,2.264820e+08,2024-12-31,1.421480e+08,2.917000e+06,1.853000e+06,108403456,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2023-09-15,True,-3.189154e+07,860.184129
7,PLSE,-5.622500e+07,1.093940e+08,2024-12-31,1.180380e+08,8.323000e+06,8.898000e+06,1135919802,Medical - Instruments & Supplies,Healthcare,https://images.financialmodelingprep.com/symbo...,2016-05-18,True,1.026780e+09,-5.475858
8,PNTG,3.748500e+07,2.115600e+07,2024-12-31,2.424600e+07,3.138820e+08,2.730910e+08,1021661160,Medical - Care Facilities,Healthcare,https://images.financialmodelingprep.com/symbo...,2019-10-01,True,1.270506e+09,2.950399


In [12]:
# Return On Capital

test_merge['returnOnCapital'] = (test_merge['ebit'] / (test_merge['propertyPlantEquipmentNet'] + test_merge['workingCapital'])) * 100

test_merge



,symbol,ebit,workingCapital,date,cashAndCashEquivalents,propertyPlantEquipmentNet,totalDebt,mktCap,industry,sector,image,ipoDate,isActivelyTrading,enterpriseValue,earningsYield,returnOnCapital
0,AACB,NaN,NaN,NaN,NaN,NaN,NaN,275919740,Shell Companies,Financial Services,https://images.financialmodelingprep.com/symbo...,2025-02-13,True,NaN,NaN,NaN
1,AAPD,NaN,NaN,NaN,NaN,NaN,NaN,32942306,Asset Management - Leveraged,Financial Services,https://images.financialmodelingprep.com/symbo...,2022-08-10,True,NaN,NaN,NaN
2,AAPL,1.273640e+11,-2.589700e+10,2024-09-28,2.994300e+10,4.568000e+10,1.066290e+11,3170272908000,Consumer Electronics,Technology,https://images.financialmodelingprep.com/symbo...,1980-12-12,True,3.246959e+12,3.922563,643.805287
3,ABLLL,-2.357387e+06,9.709857e+07,2024-12-31,1.319443e+08,5.747639e+06,3.859139e+08,842259617,Insurance - Life,Financial Services,https://images.financialmodelingprep.com/symbo...,2023-11-24,True,1.096229e+09,-0.215045,-2.292148
4,HPK,2.575150e+08,-1.462650e+08,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,1306684455,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-29,True,2.149808e+09,11.978509,9.528188
5,HPKEW,2.575150e+08,-1.462650e+08,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,106778749,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-30,True,9.499027e+08,27.109617,9.528188
6,NMRA,-2.743260e+08,2.264820e+08,2024-12-31,1.421480e+08,2.917000e+06,1.853000e+06,108403456,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2023-09-15,True,-3.189154e+07,860.184129,-119.584654
7,PLSE,-5.622500e+07,1.093940e+08,2024-12-31,1.180380e+08,8.323000e+06,8.898000e+06,1135919802,Medical - Instruments & Supplies,Healthcare,https://images.financialmodelingprep.com/symbo...,2016-05-18,True,1.026780e+09,-5.475858,-47.762855
8,PNTG,3.748500e+07,2.115600e+07,2024-12-31,2.424600e+07,3.138820e+08,2.730910e+08,1021661160,Medical - Care Facilities,Healthcare,https://images.financialmodelingprep.com/symbo...,2019-10-01,True,1.270506e+09,2.950399,11.188283


In [13]:
test_merge['rocRank'] = test_merge['returnOnCapital'].rank(ascending=False, method='min')
test_merge['yieldRank'] = test_merge['earningsYield'].rank(ascending=False, method='min')
test_merge['rankSum'] = test_merge['rocRank'] + test_merge['yieldRank']
test_merge['overallRank'] = test_merge['rankSum'].rank(ascending=True, method='min')
# test_merge['overallRank'] = test_merge['overallRank'].apply(np.int64)
test_merge.sort_values(by='overallRank', ascending=True)



,symbol,ebit,workingCapital,date,cashAndCashEquivalents,propertyPlantEquipmentNet,totalDebt,mktCap,industry,sector,image,ipoDate,isActivelyTrading,enterpriseValue,earningsYield,returnOnCapital,rocRank,yieldRank,rankSum,overallRank
2,AAPL,1.273640e+11,-2.589700e+10,2024-09-28,2.994300e+10,4.568000e+10,1.066290e+11,3170272908000,Consumer Electronics,Technology,https://images.financialmodelingprep.com/symbo...,1980-12-12,True,3.246959e+12,3.922563,643.805287,1.0,4.0,5.0,1.0
5,HPKEW,2.575150e+08,-1.462650e+08,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,106778749,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-30,True,9.499027e+08,27.109617,9.528188,3.0,2.0,5.0,1.0
4,HPK,2.575150e+08,-1.462650e+08,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,1306684455,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-29,True,2.149808e+09,11.978509,9.528188,3.0,3.0,6.0,3.0
8,PNTG,3.748500e+07,2.115600e+07,2024-12-31,2.424600e+07,3.138820e+08,2.730910e+08,1021661160,Medical - Care Facilities,Healthcare,https://images.financialmodelingprep.com/symbo...,2019-10-01,True,1.270506e+09,2.950399,11.188283,2.0,5.0,7.0,4.0
6,NMRA,-2.743260e+08,2.264820e+08,2024-12-31,1.421480e+08,2.917000e+06,1.853000e+06,108403456,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2023-09-15,True,-3.189154e+07,860.184129,-119.584654,7.0,1.0,8.0,5.0
3,ABLLL,-2.357387e+06,9.709857e+07,2024-12-31,1.319443e+08,5.747639e+06,3.859139e+08,842259617,Insurance - Life,Financial Services,https://images.financialmodelingprep.com/symbo...,2023-11-24,True,1.096229e+09,-0.215045,-2.292148,5.0,6.0,11.0,6.0
7,PLSE,-5.622500e+07,1.093940e+08,2024-12-31,1.180380e+08,8.323000e+06,8.898000e+06,1135919802,Medical - Instruments & Supplies,Healthcare,https://images.financialmodelingprep.com/symbo...,2016-05-18,True,1.026780e+09,-5.475858,-47.762855,6.0,7.0,13.0,7.0
0,AACB,NaN,NaN,NaN,NaN,NaN,NaN,275919740,Shell Companies,Financial Services,https://images.financialmodelingprep.com/symbo...,2025-02-13,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAPD,NaN,NaN,NaN,NaN,NaN,NaN,32942306,Asset Management - Leveraged,Financial Services,https://images.financialmodelingprep.com/symbo...,2022-08-10,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# remove rows where overallRank is null
test_merge.sort_values(by='overallRank', ascending=True, inplace=True)
test_merge.dropna(subset=['overallRank'])

,symbol,ebit,workingCapital,date,cashAndCashEquivalents,propertyPlantEquipmentNet,totalDebt,mktCap,industry,sector,image,ipoDate,isActivelyTrading,enterpriseValue,earningsYield,returnOnCapital,rocRank,yieldRank,rankSum,overallRank
2,AAPL,1.273640e+11,-2.589700e+10,2024-09-28,2.994300e+10,4.568000e+10,1.066290e+11,3170272908000,Consumer Electronics,Technology,https://images.financialmodelingprep.com/symbo...,1980-12-12,True,3.246959e+12,3.922563,643.805287,1.0,4.0,5.0,1.0
5,HPKEW,2.575150e+08,-1.462650e+08,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,106778749,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-30,True,9.499027e+08,27.109617,9.528188,3.0,2.0,5.0,1.0
4,HPK,2.575150e+08,-1.462650e+08,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,1306684455,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-29,True,2.149808e+09,11.978509,9.528188,3.0,3.0,6.0,3.0
8,PNTG,3.748500e+07,2.115600e+07,2024-12-31,2.424600e+07,3.138820e+08,2.730910e+08,1021661160,Medical - Care Facilities,Healthcare,https://images.financialmodelingprep.com/symbo...,2019-10-01,True,1.270506e+09,2.950399,11.188283,2.0,5.0,7.0,4.0
6,NMRA,-2.743260e+08,2.264820e+08,2024-12-31,1.421480e+08,2.917000e+06,1.853000e+06,108403456,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2023-09-15,True,-3.189154e+07,860.184129,-119.584654,7.0,1.0,8.0,5.0
3,ABLLL,-2.357387e+06,9.709857e+07,2024-12-31,1.319443e+08,5.747639e+06,3.859139e+08,842259617,Insurance - Life,Financial Services,https://images.financialmodelingprep.com/symbo...,2023-11-24,True,1.096229e+09,-0.215045,-2.292148,5.0,6.0,11.0,6.0
7,PLSE,-5.622500e+07,1.093940e+08,2024-12-31,1.180380e+08,8.323000e+06,8.898000e+06,1135919802,Medical - Instruments & Supplies,Healthcare,https://images.financialmodelingprep.com/symbo...,2016-05-18,True,1.026780e+09,-5.475858,-47.762855,6.0,7.0,13.0,7.0


In [21]:
# inspect clean data for further cleaning
cleanish_data = pd.read_csv('../data/clean/clean_ranked_data.csv')
cleanish_data

,date,symbol,cashAndCashEquivalents,propertyPlantEquipmentNet,totalDebt,mktCap,industry,sector,image,ipoDate,...,workingCapital,Security Name,Exchange,enterpriseValue,earningsYield,returnOnCapital,rocRank,yieldRank,rankSum,overallRank
0,2024-12-31,BLW,1865000.0,0.0,0.0,5.040487e+08,Asset Management - Income,Financial Services,https://images.financialmodelingprep.com/symbo...,2003-07-29,...,2211005.0,Blackrock Limited Duration Income Trust,NYSE,5.021837e+08,2901.574079,6.590321e+05,121.0,25.0,146.0,1.0
1,2024-12-31,MITN,118662000.0,0.0,95721000.0,2.026894e+08,REIT - Mortgage,Real Estate,https://images.financialmodelingprep.com/symbo...,2024-02-06,...,0.0,"AG Mortgage Investment Trust, Inc. 9.500% Seni...",NYSE,1.797484e+08,190.018376,inf,1.0,164.0,165.0,2.0
2,2024-12-31,ACONW,453661.0,5499.0,0.0,2.090700e+04,Medical - Healthcare Information Services,Healthcare,https://images.financialmodelingprep.com/symbo...,NaN,...,-329147.0,"Aclarion, Inc. - Warrant",NASDAQ,-4.327540e+05,1492.179160,1.995212e+03,229.0,41.0,270.0,3.0
3,2024-12-31,INBX,152596000.0,13538000.0,8048000.0,1.695128e+08,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2024-06-04,...,119668000.0,"Inhibrx Biosciences, Inc. - Common Stock",NASDAQ,2.496484e+07,6800.841792,1.274581e+03,272.0,8.0,280.0,4.0
4,2024-11-30,KYN,0.0,0.0,0.0,2.107310e+09,Asset Management,Financial Services,https://images.financialmodelingprep.com/symbo...,2004-09-28,...,0.0,"Kayne Anderson Energy Infrastructure Fund, Inc.",NYSE,2.107310e+09,46.455292,inf,1.0,306.0,307.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6014,2023-12-31,ADNWW,3562000.0,24765000.0,10416000.0,2.346500e+04,Renewable Utilities,Utilities,https://images.financialmodelingprep.com/symbo...,2019-01-24,...,-23720000.0,"Advent Technologies Holdings, Inc. - Warrant",NASDAQ,6.877465e+06,-698.295069,-4.595694e+03,5979.0,5927.0,11906.0,6015.0
6015,2024-12-31,CING,12211321.0,2203686.0,5099079.0,1.689794e+07,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2021-12-08,...,7688698.0,Cingulate Inc. - Common Stock,NASDAQ,9.785696e+06,-1708.814498,-1.690385e+03,5945.0,5985.0,11930.0,6016.0
6016,2024-03-31,PRTG,5028000.0,35000.0,47000.0,8.531259e+06,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2002-01-02,...,1813000.0,Portage Biotech Inc. - Common Stock,NASDAQ,3.550259e+06,-2199.445167,-4.225433e+03,5976.0,5993.0,11969.0,6017.0
6017,2024-12-31,VLCN,2193573.0,945372.0,810886.0,2.849610e+06,Auto - Manufacturers,Consumer Cyclical,https://images.financialmodelingprep.com/symbo...,2021-10-06,...,429349.0,"Volcon, Inc. - Common stock",NASDAQ,1.466923e+06,-3050.221518,-3.254799e+03,5972.0,6001.0,11973.0,6018.0


In [ ]:
cash_zero_count = (cleanish_data['cashAndCashEquivalents'] == 0).sum()
property_zeros = (cleanish_data['propertyPlantEquipmentNet'] == 0).sum()
debt_zeros = (cleanish_data['totalDebt'] == 0.0).sum()
non_active = (cleanish_data['isActivelyTrading'] == False).sum()
capital_zeros = (cleanish_data['workingCapital'] == 0).sum()
ev_zeros = (cleanish_data['enterpriseValue'] == 0).sum()

print(cash_zero_count)
print(property_zeros)
print(debt_zeros)
print(non_active)
print(capital_zeros)
print(ev_zeros)


184
960
557
58
265
0


In [28]:
cols_to_check = ['cashAndCashEquivalents', 'propertyPlantEquipmentNet', 'totalDebt', 'isActivelyTrading', 'workingCapital', 'enterpriseValue']
cleanish_data = cleanish_data[~(cleanish_data[cols_to_check] == 0).any(axis=1)]
cleanish_data

,date,symbol,cashAndCashEquivalents,propertyPlantEquipmentNet,totalDebt,mktCap,industry,sector,image,ipoDate,...,workingCapital,Security Name,Exchange,enterpriseValue,earningsYield,returnOnCapital,rocRank,yieldRank,rankSum,overallRank
3,2024-12-31,INBX,152596000.0,13538000.0,8048000.0,169512836.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2024-06-04,...,119668000.0,"Inhibrx Biosciences, Inc. - Common Stock",NASDAQ,24964836.0,6800.841792,1274.581475,272.0,8.0,280.0,4.0
5,2024-12-31,PRFX,4261000.0,97000.0,45000.0,3638050.0,Drug Manufacturers - Specialty & Generic,Healthcare,https://images.financialmodelingprep.com/symbo...,2020-09-01,...,-1899000.0,PainReform Ltd. - Ordinary Shares,NASDAQ,-577950.0,3084.349857,989.234184,305.0,21.0,326.0,6.0
15,2024-12-31,CEROW,3327060.0,1992888.0,1575499.0,93724.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,NaN,...,-3688344.0,"CERo Therapeutics Holdings, Inc. - Warrants",NASDAQ,-1657837.0,638.146151,623.986880,364.0,77.0,441.0,16.0
32,2024-12-31,MBIO,6839000.0,452000.0,878000.0,3735355.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2017-08-22,...,-4003000.0,"Mustang Bio, Inc. - Common Stock",NASDAQ,-2225645.0,707.525234,443.452549,420.0,72.0,492.0,33.0
40,2024-12-31,PAL,15398714.0,133607172.0,93571545.0,222237434.0,Integrated Freight & Logistics,Industrials,https://images.financialmodelingprep.com/symbo...,2012-02-20,...,15063511.0,"Proficient Auto Logistics, Inc. - Common Stock",NASDAQ,300410265.0,256.776628,518.853706,390.0,130.0,520.0,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6014,2023-12-31,ADNWW,3562000.0,24765000.0,10416000.0,23465.0,Renewable Utilities,Utilities,https://images.financialmodelingprep.com/symbo...,2019-01-24,...,-23720000.0,"Advent Technologies Holdings, Inc. - Warrant",NASDAQ,6877465.0,-698.295069,-4595.693684,5979.0,5927.0,11906.0,6015.0
6015,2024-12-31,CING,12211321.0,2203686.0,5099079.0,16897938.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2021-12-08,...,7688698.0,Cingulate Inc. - Common Stock,NASDAQ,9785696.0,-1708.814498,-1690.385169,5945.0,5985.0,11930.0,6016.0
6016,2024-03-31,PRTG,5028000.0,35000.0,47000.0,8531259.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2002-01-02,...,1813000.0,Portage Biotech Inc. - Common Stock,NASDAQ,3550259.0,-2199.445167,-4225.432900,5976.0,5993.0,11969.0,6017.0
6017,2024-12-31,VLCN,2193573.0,945372.0,810886.0,2849610.0,Auto - Manufacturers,Consumer Cyclical,https://images.financialmodelingprep.com/symbo...,2021-10-06,...,429349.0,"Volcon, Inc. - Common stock",NASDAQ,1466923.0,-3050.221518,-3254.798683,5972.0,6001.0,11973.0,6018.0


In [29]:
df = pd.read_csv('../data/clean/clean_ranked_data1.csv')
df

,date,symbol,cashAndCashEquivalents,propertyPlantEquipmentNet,totalDebt,mktCap,industry,sector,image,ipoDate,...,workingCapital,Security Name,Exchange,enterpriseValue,earningsYield,returnOnCapital,rocRank,yieldRank,rankSum,overallRank
0,2024-12-31,INBX,152596000.0,13538000.0,8048000.0,169512836.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2024-06-04,...,119668000.0,"Inhibrx Biosciences, Inc. - Common Stock",NASDAQ,24964836.0,6800.841792,1274.581475,36.0,5.0,41.0,1.0
1,2024-12-31,PRFX,4261000.0,97000.0,45000.0,3638050.0,Drug Manufacturers - Specialty & Generic,Healthcare,https://images.financialmodelingprep.com/symbo...,2020-09-01,...,-1899000.0,PainReform Ltd. - Ordinary Shares,NASDAQ,-577950.0,3084.349857,989.234184,48.0,15.0,63.0,2.0
2,2024-12-31,CEROW,3327060.0,1992888.0,1575499.0,93724.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,NaN,...,-3688344.0,"CERo Therapeutics Holdings, Inc. - Warrants",NASDAQ,-1657837.0,638.146151,623.986880,74.0,59.0,133.0,3.0
3,2024-12-31,MBIO,6839000.0,452000.0,878000.0,3735355.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2017-08-22,...,-4003000.0,"Mustang Bio, Inc. - Common Stock",NASDAQ,-2225645.0,707.525234,443.452549,98.0,55.0,153.0,4.0
4,2024-12-31,PAL,15398714.0,133607172.0,93571545.0,222237434.0,Integrated Freight & Logistics,Industrials,https://images.financialmodelingprep.com/symbo...,2012-02-20,...,15063511.0,"Proficient Auto Logistics, Inc. - Common Stock",NASDAQ,300410265.0,256.776628,518.853706,84.0,99.0,183.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4688,2024-12-31,RYDE,5519000.0,155000.0,135000.0,6309557.0,Software - Application,Technology,https://images.financialmodelingprep.com/symbo...,2024-03-06,...,3063000.0,Ryde Group Ltd. Class A Ordinary Shares,AMEX,925557.0,-3014.364233,-866.987544,4627.0,4679.0,9306.0,4689.0
4689,2024-12-31,CING,12211321.0,2203686.0,5099079.0,16897938.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2021-12-08,...,7688698.0,Cingulate Inc. - Common Stock,NASDAQ,9785696.0,-1708.814498,-1690.385169,4661.0,4666.0,9327.0,4690.0
4690,2024-03-31,PRTG,5028000.0,35000.0,47000.0,8531259.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2002-01-02,...,1813000.0,Portage Biotech Inc. - Common Stock,NASDAQ,3550259.0,-2199.445167,-4225.432900,4673.0,4673.0,9346.0,4691.0
4691,2024-12-31,VLCN,2193573.0,945372.0,810886.0,2849610.0,Auto - Manufacturers,Consumer Cyclical,https://images.financialmodelingprep.com/symbo...,2021-10-06,...,429349.0,"Volcon, Inc. - Common stock",NASDAQ,1466923.0,-3050.221518,-3254.798683,4671.0,4680.0,9351.0,4692.0


In [30]:
df = pd.read_csv('../data/clean/clean_ranked_data2.csv')
df

,date,symbol,cashAndCashEquivalents,propertyPlantEquipmentNet,totalDebt,mktCap,industry,sector,image,ipoDate,...,workingCapital,Security Name,Exchange,enterpriseValue,earningsYield,returnOnCapital,rocRank,yieldRank,rankSum,overallRank
0,2024-12-31,INBX,152596000.0,13538000.0,8048000.0,169512836.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2024-06-04,...,119668000.0,"Inhibrx Biosciences, Inc. - Common Stock",NASDAQ,24964836.0,6800.841792,1274.581475,36.0,5.0,41.0,1.0
1,2024-12-31,PRFX,4261000.0,97000.0,45000.0,3638050.0,Drug Manufacturers - Specialty & Generic,Healthcare,https://images.financialmodelingprep.com/symbo...,2020-09-01,...,-1899000.0,PainReform Ltd. - Ordinary Shares,NASDAQ,-577950.0,3084.349857,989.234184,48.0,15.0,63.0,2.0
2,2024-12-31,CEROW,3327060.0,1992888.0,1575499.0,93724.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,NaN,...,-3688344.0,"CERo Therapeutics Holdings, Inc. - Warrants",NASDAQ,-1657837.0,638.146151,623.986880,74.0,59.0,133.0,3.0
3,2024-12-31,MBIO,6839000.0,452000.0,878000.0,3735355.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2017-08-22,...,-4003000.0,"Mustang Bio, Inc. - Common Stock",NASDAQ,-2225645.0,707.525234,443.452549,98.0,55.0,153.0,4.0
4,2024-12-31,PAL,15398714.0,133607172.0,93571545.0,222237434.0,Integrated Freight & Logistics,Industrials,https://images.financialmodelingprep.com/symbo...,2012-02-20,...,15063511.0,"Proficient Auto Logistics, Inc. - Common Stock",NASDAQ,300410265.0,256.776628,518.853706,84.0,99.0,183.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4688,2024-12-31,RYDE,5519000.0,155000.0,135000.0,6309557.0,Software - Application,Technology,https://images.financialmodelingprep.com/symbo...,2024-03-06,...,3063000.0,Ryde Group Ltd. Class A Ordinary Shares,AMEX,925557.0,-3014.364233,-866.987544,4627.0,4679.0,9306.0,4689.0
4689,2024-12-31,CING,12211321.0,2203686.0,5099079.0,16897938.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2021-12-08,...,7688698.0,Cingulate Inc. - Common Stock,NASDAQ,9785696.0,-1708.814498,-1690.385169,4661.0,4666.0,9327.0,4690.0
4690,2024-03-31,PRTG,5028000.0,35000.0,47000.0,8531259.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2002-01-02,...,1813000.0,Portage Biotech Inc. - Common Stock,NASDAQ,3550259.0,-2199.445167,-4225.432900,4673.0,4673.0,9346.0,4691.0
4691,2024-12-31,VLCN,2193573.0,945372.0,810886.0,2849610.0,Auto - Manufacturers,Consumer Cyclical,https://images.financialmodelingprep.com/symbo...,2021-10-06,...,429349.0,"Volcon, Inc. - Common stock",NASDAQ,1466923.0,-3050.221518,-3254.798683,4671.0,4680.0,9351.0,4692.0


In [33]:
df = pd.read_csv('../data/clean/clean_ranked_data4.csv')
df

,date,symbol,cashAndCashEquivalents,propertyPlantEquipmentNet,totalDebt,mktCap,industry,sector,image,isActivelyTrading,...,workingCapital,Security Name,Exchange,enterpriseValue,earningsYield,returnOnCapital,rocRank,yieldRank,rankSum,overallRank
0,2024-12-31,INBX,152596000.0,13538000.0,8048000.0,169512836.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,True,...,119668000.0,"Inhibrx Biosciences, Inc. - Common Stock",NASDAQ,24964836.0,6800.841792,1274.581475,36.0,5.0,41.0,1.0
1,2024-12-31,PRFX,4261000.0,97000.0,45000.0,3638050.0,Drug Manufacturers - Specialty & Generic,Healthcare,https://images.financialmodelingprep.com/symbo...,True,...,-1899000.0,PainReform Ltd. - Ordinary Shares,NASDAQ,-577950.0,3084.349857,989.234184,48.0,15.0,63.0,2.0
2,2024-12-31,CEROW,3327060.0,1992888.0,1575499.0,93724.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,True,...,-3688344.0,"CERo Therapeutics Holdings, Inc. - Warrants",NASDAQ,-1657837.0,638.146151,623.986880,74.0,59.0,133.0,3.0
3,2024-12-31,MBIO,6839000.0,452000.0,878000.0,3735355.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,True,...,-4003000.0,"Mustang Bio, Inc. - Common Stock",NASDAQ,-2225645.0,707.525234,443.452549,98.0,55.0,153.0,4.0
4,2024-12-31,PAL,15398714.0,133607172.0,93571545.0,222237434.0,Integrated Freight & Logistics,Industrials,https://images.financialmodelingprep.com/symbo...,True,...,15063511.0,"Proficient Auto Logistics, Inc. - Common Stock",NASDAQ,300410265.0,256.776628,518.853706,84.0,99.0,183.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4688,2024-12-31,RYDE,5519000.0,155000.0,135000.0,6309557.0,Software - Application,Technology,https://images.financialmodelingprep.com/symbo...,True,...,3063000.0,Ryde Group Ltd. Class A Ordinary Shares,AMEX,925557.0,-3014.364233,-866.987544,4627.0,4679.0,9306.0,4689.0
4689,2024-12-31,CING,12211321.0,2203686.0,5099079.0,16897938.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,True,...,7688698.0,Cingulate Inc. - Common Stock,NASDAQ,9785696.0,-1708.814498,-1690.385169,4661.0,4666.0,9327.0,4690.0
4690,2024-03-31,PRTG,5028000.0,35000.0,47000.0,8531259.0,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,True,...,1813000.0,Portage Biotech Inc. - Common Stock,NASDAQ,3550259.0,-2199.445167,-4225.432900,4673.0,4673.0,9346.0,4691.0
4691,2024-12-31,VLCN,2193573.0,945372.0,810886.0,2849610.0,Auto - Manufacturers,Consumer Cyclical,https://images.financialmodelingprep.com/symbo...,True,...,429349.0,"Volcon, Inc. - Common stock",NASDAQ,1466923.0,-3050.221518,-3254.798683,4671.0,4680.0,9351.0,4692.0
